In [0]:
CREATE OR REFRESH STREAMING TABLE gold_revenue_summary
COMMENT "Aggregated Zomato revenue data for analytics"
TBLPROPERTIES ("quality" = "gold")
AS
SELECT
    City,
    Restaurant,
    Category,
    DATE_TRUNC('month', OrderDate) AS Month,
    COUNT(OrderID) AS TotalOrders,
    SUM(ItemTotal) AS TotalItemRevenue,
    SUM(DeliveryFee) AS TotalDeliveryFee,
    SUM(Taxes) AS TotalTaxes,
    SUM(Discount) AS TotalDiscount,
    SUM(TotalRevenue) AS NetRevenue,
    ROUND(AVG(TotalRevenue), 2) AS AvgOrderValue
FROM STREAM(silver_revenue)
GROUP BY
    City,
    Restaurant,
    Category,
    DATE_TRUNC('month', OrderDate);

In [0]:
CREATE OR REFRESH STREAMING TABLE gold_customer_restaurant_metrics
COMMENT "Aggregated customer and restaurant insights for analytics (without customer_segment and most_recent_category)"
TBLPROPERTIES ("quality" = "gold")
AS
SELECT
    c.customer_id,
    c.full_name,
    c.city AS customer_city,
    c.gender,
    c.age,
    r.RestaurantId,
    r.Restaurant_Name,
    r.City AS restaurant_city,
    r.Cuisine_type,
    r.Rating AS restaurant_rating,
    DATE_TRUNC('month', o.order_datetime) AS Month,
    approx_count_distinct(o.OrderId) AS total_orders,
    SUM(o.Order_amount) AS total_order_amount,
    SUM(o.tax) AS total_tax,
    SUM(o.discount) AS total_discount,
    SUM(o.delivery_fee) AS total_delivery_fee,
    AVG(op.Delivery_time) AS avg_delivery_time,
    AVG(op.Delivery_distance) AS avg_delivery_distance,
    approx_count_distinct(oi.ItemId) AS unique_items_ordered,
    AVG(oi.Quantity) AS avg_item_quantity,
    AVG(mi.Price) AS avg_menu_price,
    c.total_orders AS lifetime_orders,
    c.total_spent AS lifetime_spent
FROM STREAM(LIVE.silver_orders) AS o
LEFT JOIN LIVE.silver_customers AS c ON o.CustomerId = c.customer_id
LEFT JOIN LIVE.silver_restaurants AS r ON o.RestaurantId = r.RestaurantId
LEFT JOIN LIVE.silver_operations AS op ON o.OrderId = op.OrderId
LEFT JOIN LIVE.silver_orderItems AS oi ON o.OrderId = oi.OrderId
LEFT JOIN LIVE.silver_menuitems AS mi ON oi.ItemId = mi.ItemId
WHERE o.order_status != 'Cancelled'
GROUP BY c.customer_id, c.full_name, c.city, c.gender, c.age,
         r.RestaurantId, r.Restaurant_Name, r.City, r.Cuisine_type, r.Rating,
